In [ ]:
class TrieNode:
    def __init__(self):
        self.children: TrieNode = [None] * 26  # Assuming only lowercase English letters
        self.isEndOfWord = False

    def __repr__(self):
        # Show which characters this node has as children and if it's the end of a word
        children_indices = [chr(i + ord('a')) for i, child in enumerate(self.children) if child]
        return f"TrieNode(children: {children_indices}, isEndOfWord: {self.isEndOfWord})"


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def __repr__(self):
        # Only show a summary of the root node's children
        root_children = [chr(i + ord('a')) for i, child in enumerate(self.root.children) if child]
        return f"Trie(root: TrieNode(children: {root_children}))"
    
    def charToIndex(self, ch: str):
        # Convert character to index (0-25)
        return ord(ch) - ord('a')

    def insert(self, word: str):
        node = self.root
        for char in word:
            index = self.charToIndex(char)
            if not node.children[index]:
                node.children[index] = TrieNode()
            node = node.children[index]
        node.isEndOfWord = True

    # Function to search a word in the Trie
    def search(self, word: str):
        node = self.root
        for char in word:
            index = ord(char) - ord('a')
            if not node.children[index]:
                return False  # Word not found
            node = node.children[index]
        return node.isEndOfWord  # Return true if word exists, false otherwise


    def delete(self, word: str):
        word_exists = self.search(word)
        if word_exists:
            # Initiate the recursive deletion process
            self._deleteRecursively(self.root, word, 0)
        return word_exists

    def _deleteRecursively(self, current_node: TrieNode, word: str, index: int) -> bool:
        """
        Recursively descend into the trie until the end of the word is reached.
        Returns True if the current node can be deleted from its parent.
        """
        # 1. Base Case (Termination Condition): If we've reached the end of the word
        is_end_of_word = (index == len(word))
        if is_end_of_word:
            return self._handleEndOfWordDeletion(current_node)
        
        # 2. Pre-Processing (Preparation and Validation): Ensure the child node exists before continuing

        current_character = word[index]
        child_node = self._getChildNode(current_node, current_character)
        if not child_node:
            return False

        # 3. Recursive Case (Reduction Step): Call recursively on the next node
        should_delete_child = self._deleteRecursively(child_node, word, index + 1)

        # 4. Post-Processing (Combining Results): Remove child node if necessary
        if should_delete_child:
            self._removeChildNode(current_node, current_character)
            return self._shouldDeleteCurrentNode(current_node)

        return False


    def _handleEndOfWordDeletion(self, node: TrieNode) -> bool:
        """
        Unmark the end-of-word from the current node.
        Decide if the node can be deleted after unmarking.
        """
        self._unmarkEndOfWord(node)
        return self._shouldDeleteCurrentNode(node)

    def _unmarkEndOfWord(self, node: TrieNode):
        """Unmark the end-of-word flag for this node."""
        node.isEndOfWord = False

    def _shouldDeleteCurrentNode(self, node: TrieNode) -> bool:
        """
        Determine if the current node should be deleted.
        A node should be deleted if it's not the end of another word and has no children.
        """
        no_children = not any(node.children)
        return not node.isEndOfWord and no_children

    def _getChildNode(self, node: TrieNode, char: str) -> TrieNode:
        """Retrieve the child node for the given character, if it exists."""
        return node.children[self.charToIndex(char)]

    def _removeChildNode(self, node: TrieNode, char: str):
        """Remove the child node corresponding to the given character."""
        node.children[self.charToIndex(char)] = None



In [37]:
def main():
    # Create a Trie object
    trie = Trie()
    
    # Words to insert
    words_to_insert = ["hello", "world", "trie", "tree", "algorithm"]
    
    # Insert words into the Trie
    print("Inserting words into the Trie:")
    for word in words_to_insert:
        trie.insert(word)
        print(f"Inserted: {word}")
    
    print("\nSearching for words in the Trie:")
    search_words = ["hello", "world", "trie", "tree", "algorithms", "data"]
    for word in search_words:
        found = trie.search(word)
        print(f"Word '{word}' found: {found}")
    
    print("\nDeleting words from the Trie:")
    delete_words = ["world", "trie"]
    for word in delete_words:
        deleted = trie.delete(word)
        print(f"Word '{word}' deleted: {deleted}")
    
    print("\nSearching again after deletion:")
    for word in search_words:
        found = trie.search(word)
        print(f"Word '{word}' found: {found}")

if __name__ == "__main__":
    main()


Inserting words into the Trie:
Inserted: hello
Inserted: world
Inserted: trie
Inserted: tree
Inserted: algorithm

Searching for words in the Trie:
Word 'hello' found: True
Word 'world' found: True
Word 'trie' found: True
Word 'tree' found: True
Word 'algorithms' found: False
Word 'data' found: False

Deleting words from the Trie:
Word 'world' deleted: True
Word 'trie' deleted: True

Searching again after deletion:
Word 'hello' found: True
Word 'world' found: False
Word 'trie' found: False
Word 'tree' found: True
Word 'algorithms' found: False
Word 'data' found: False


In [38]:
# Create a Trie instance
trie = Trie()

# Test Case 1: Delete a single word
trie.insert("cat")
print("Test 1: Deleting a single word")
print("Before delete, 'cat' exists:", trie.search("cat"))  # Expected: True
print("Delete 'cat':", trie.delete("cat"))                # Expected: True
print("After delete, 'cat' exists:", trie.search("cat"))  # Expected: False
print()

# Test Case 2: Delete a word not in the Trie
print("Test 2: Deleting a word not in the Trie")
print("Before delete, 'dog' exists:", trie.search("dog"))  # Expected: False
print("Delete 'dog':", trie.delete("dog"))                # Expected: False
print("After delete, 'dog' exists:", trie.search("dog"))  # Expected: False
print()

# Test Case 3: Delete a word that is a prefix of another word
trie.insert("cat")
trie.insert("cater")
print("Test 3: Deleting a word that is a prefix of another word")
print("Before delete, 'cat' exists:", trie.search("cat"))    # Expected: True
print("Before delete, 'cater' exists:", trie.search("cater"))# Expected: True
print("Delete 'cat':", trie.delete("cat"))                  # Expected: True
print("After delete, 'cat' exists:", trie.search("cat"))    # Expected: False
print("After delete, 'cater' exists:", trie.search("cater"))# Expected: True
print()

# Test Case 4: Delete a word that has a prefix in the Trie
trie.insert("cat")
trie.insert("cater")
print("Test 4: Deleting a word that has a prefix in the Trie")
print("Before delete, 'cat' exists:", trie.search("cat"))    # Expected: True
print("Before delete, 'cater' exists:", trie.search("cater"))# Expected: True
print("Delete 'cater':", trie.delete("cater"))              # Expected: True
print("After delete, 'cat' exists:", trie.search("cat"))    # Expected: True
print("After delete, 'cater' exists:", trie.search("cater"))# Expected: False
print()

# Test Case 5: Delete a word not in the Trie but with shared prefix
trie.insert("car")
trie.insert("cargo")
print("Test 5: Deleting a word not in the Trie but with shared prefix")
print("Before delete, 'cart' exists:", trie.search("cart"))  # Expected: False
print("Delete 'cart':", trie.delete("cart"))                # Expected: False
print("After delete, 'car' exists:", trie.search("car"))    # Expected: True
print("After delete, 'cargo' exists:", trie.search("cargo"))# Expected: True
print()

# Test Case 6: Complex scenario with multiple shared prefixes
trie.insert("bat")
trie.insert("batch")
trie.insert("batter")
print("Test 6: Complex scenario with multiple shared prefixes")
print("Before delete, 'bat' exists:", trie.search("bat"))      # Expected: True
print("Before delete, 'batch' exists:", trie.search("batch"))  # Expected: True
print("Before delete, 'batter' exists:", trie.search("batter"))# Expected: True
print("Delete 'batch':", trie.delete("batch"))                # Expected: True
print("After delete, 'bat' exists:", trie.search("bat"))      # Expected: True
print("After delete, 'batch' exists:", trie.search("batch"))  # Expected: False
print("After delete, 'batter' exists:", trie.search("batter"))# Expected: True
print()


Test 1: Deleting a single word
Before delete, 'cat' exists: True
Delete 'cat': True
After delete, 'cat' exists: False

Test 2: Deleting a word not in the Trie
Before delete, 'dog' exists: False
Delete 'dog': False
After delete, 'dog' exists: False

Test 3: Deleting a word that is a prefix of another word
Before delete, 'cat' exists: True
Before delete, 'cater' exists: True
Delete 'cat': True
After delete, 'cat' exists: False
After delete, 'cater' exists: True

Test 4: Deleting a word that has a prefix in the Trie
Before delete, 'cat' exists: True
Before delete, 'cater' exists: True
Delete 'cater': True
After delete, 'cat' exists: True
After delete, 'cater' exists: False

Test 5: Deleting a word not in the Trie but with shared prefix
Before delete, 'cart' exists: False
Delete 'cart': False
After delete, 'car' exists: True
After delete, 'cargo' exists: True

Test 6: Complex scenario with multiple shared prefixes
Before delete, 'bat' exists: True
Before delete, 'batch' exists: True
Before